(Intermediate_States)=
# Constructing a Pathway of Intermediate States

Because the phase space overlap between two target states of interest can be near zero, doing free energy calculations for the two states alone will often have very large errors. When one considers the large number of potential molecules that could be considered interesting, this problem becomes even more pronounced. Because free energy is a state function, we can construct a [[Thermodynamic_Cycle|thermodynamic path]] that takes us through a set of states that improves phase space overlap between states. To put this mathematically, we can improve our results by constructing high phase space overlap intermediates and calculating our free energy difference by

$\Delta A_{1,K} = \sum_{i=1}^{K-1} \Delta A_{i,i+1}$.

One advantage available to computational free energy calculations is the ability to simulate unphysical states. By this, we mean that our **intermediate states do not have to be observable experimentally.** This is a fact that most if not all computational methods out there take advantage of. Again, because free energy is a state function, we simply choose the path of greatest convenience and carry out the calculation across this path. However, choosing the "correct" or even a "good" path is a very challenging action and is one of the most difficult tasks in the entire field of free energy calculations.

Because our free energy calculations frequently involve transforming one kind of atom at a given site to another kind, the transformations are often referred to as "alchemical."

## Linear Alchemical Potential
To be more clear about what it means to define an "alchemical path," one should think of it as defining the thermodynamic path where we modify, remove, or add various forces on an atom. Take for instance estimating the free energy difference between a Lennard-Jones fluid and a Stockmayer fluid, our thermodynamic path would calculate the work and free energy required to switch on the dipole interactions at a rate that maximizes phase space overlap and efficiency.{{cite|Frenkel2002}} Because the atoms changed their interactions with the surroundings without being removed or added from the system, we have directly modified the atoms to create our alchemical path.

Most alchemical transformations can be defined by alchemically scaling the potential in some manner. The simplest of these is the linear transformation, which says that the net potential energy, $U(\lambda,\vec{q})$, is the sum of the alchemically modified two end states' potentials, $U_0$ and $U_1$, plus the parts of the potential that are unaffected by the alchemical transformation, $U_{\mathrm{unaffected}}$; or

$U(\lambda,\vec{q}) = (1-\lambda) U_0(\vec{q}) + \lambda U_1(\vec{q}) + U_{\mathrm{unaffected}}(\vec{q})$

where the alchemical variable linearly modifies the confrontational information from each state of interest.

### Problems with Linear Transformations
```{figure} ../images/Linear_LJ_transformation.png
---
width: 400px
name: linearXform
---
Linearly scaled Lennard-Jones potential. Note that even with very small $\lambda$, the singularity at $r=0$ is still very much there. Image recreated based on Beutler *et. al.*{{cite|Beutler1994}}
```
The largest problem with linear scaling is that it **gives very unequal phase space overlap** between states. This has been a long acknowledged problem with several easily identifiable causes:

1. Consider the OPLS-AA force field, at $\lambda=0.1$, the excluded volume of a methane sphere is still 60-70%.
1. Most potential energy functions have an infinite potential (singularity) at $r=0$. This is usually brought on by the $r^{-12}$ term that appears in Lennard-Jones based potentials and is present at all nonzero values of $\lambda$. Since most the free energy techniques require some numeric methods, evaluating this singularity diverges the calculations.

This choice also leads to large forces, numerical instabilities, and other problems in simulations near $\lambda=1$. Formally, it has been shown that this leads to a integrable singularity in $dV/(d\lambda)$, which means that computing correct free energies with this scheme using thermodynamic integration is impossible using numerical techniques{{cite|Mruzik1976}}{{cite|Mezei1986}}{{cite|Resat1993}} including with more modern methods{{cite|Beutler1994}}{{cite|Pitera2002}}{{cite|Steinbrecher2007}} and similar problems plague free energy perturbation schemes.

This singularity causes the most problems with [[Thermodynamic Integration | TI ]] since that method requires evaluation {{#tag:math| {{dudl}} }} numerically, which diverges at the singularity, but the same issues result in large biases for BAR and MBAR as well, because the phase space volumes do not overlap.

Several workarounds for this have been suggested. One of the simplest is to raise the $\lambda$ terms to a power greater than or equal to 4 like so:

$\displaystyle U(\lambda) = (1-\lambda)^4 U_0 + \lambda^4 U_1$

this leads to an integrable singularity in $dV/d\lambda$, so thermodynamic integration can in principle be done{{cite|Mezei1986}}{{cite|Beutler1994}}. But integrable singularities still pose very substantial problems for molecular simulation, and this approach can still lead to large forces, numerical instabilities and energy conservation problems{{cite|Beutler1994}}{{cite|Steinbrecher2007}} and make free energy differences extremely difficult to converge (D. Mobley, unpub. data).

These methods do converge {{#tag:math| \left\langle{{dudl}}\right\rangle }} however very slowly  with number of samples and still can cause numeric instabilities{{cite|Pitera2002}}{{cite|Steinbrecher2007}}

Other methods have been suggested such as shrinking the core of the atom, but this also causes instabilities{{cite|Pitera2002}}{{cite|Pearlman1995}}{{cite|Wang1994}} and is not practical for systems with many bonds.

## Soft Core Potentials
A standard method has been developed to alchemically change molecules to get around the numeric instabilities called a "soft core potential."{{cite|Beutler1994}}{{cite|Zacharias1994}} In these potentials, the configuration variable, $r$, is now coupled to the alchemical variable, $\lambda$, smoothing out the singularity and looks like

$U(\lambda,r) = 4\epsilon\lambda^n \left[\left(\alpha(1-\lambda)^m + \left(\frac{r}{\sigma}\right)^6\right)^{-2}  - \left(\alpha(1-\lambda)^m + \left(\frac{r}{\sigma}\right)^6\right)^{-1}\right]$

where $\alpha$ is a constant (usually 0.5) and, $m$ and $n$ are positive integers with the original choice being $n=4$ and $m=2$. Recent improvements have shown that keeping $\alpha=0.5$ and setting $m=n=1$ provides significant improvement to the variance {{cite|Pitera2002}} {{cite|Steinbrecher2007}}{{cite|Shirts2005}} and research into further improving this is still going on.{{cite|Pham2001}}

This potential is now considered the standard for avoiding endpoint errors, and some version of this is common in many software packages that support free energy calculations  As a [[Theoretic Principals#|naming convention]], when "soft core" is referenced, it refers to the this potential where as "linear" refers to the [[#Linear Alchemical Potential | linear alchemical potential]] where the exponent is 1 (i.e. $\lambda$ *not* $\lambda^4$).

In summary: Linearly scaling Lennard-Jones interactions back as a function of $\lambda$ for insertion/deletion of particles is formally incorrect for numerical integration and leads to wrong estimates of free energy differences. While more complicated schemes involving $\lambda^k$ scaling can be formally correct, there are serious concerns regarding their accuracy. Soft-core potentials provide a rigorously correct, efficient alternative to these and should be used whenever particles are inserted or deleted, preferably with a specific functional form and parameters{{cite|Shirts2005}}, unless future work finds a still more efficient set of parameters.

## Avoid turning off charges and Lennard-Jones sites at the same time

Guideline 2 states that a partial atomic charge should never be allowed to remain on an atom while its Lennard-Jones interactions are being removed. To understand the reason for this, consider two atoms of opposite charge, A and B. Lennard-Jones interactions of atom A are being scaled back. Regardless of the scaling scheme used, at some lambda value, atoms A and B will begin to overlap occasionally, since the final state allows A and B to overlap totally. If A has a remaining partial atomic charge when these overlaps become possible, the two point charges assigned to A and B can actually overlap as well. Since the potential energy of Coulomb interactions between point charges scales as $q_{A} q_{B}/r_{AB}$, where $r_{AB}$ is the distance between A and B, this presents significant problems when $q_A$ and $q_B$ have opposite signs. In particular, there is an infinite energy minimum at $r_{AB}=0$, so the two particles would in principle get trapped on top of one another. 

In practice, what usually happens in molecular dynamics simulations in these circumstances is that the forces get extremely large as A and B begin to overlap, and the simulation will crash. Constraint algorithms are often the first to fail, so this may lead to a warning about constraints (i.e. LINCS or SHAKE) and then a crash. This issue is discussed briefly by Pitera and van Gunsteren{{cite|Pitera2002}} and in more detail by Anwar and Heyes{{cite|Anwar2005}}.

In view of this problem, we recommend always turning off partial charges for any atoms for which Lennard-Jones interactions are being removed before doing the Lennard-Jones transformation. Additionally, when Lennard-Jones parameters for an atom are being substantially modified during a free energy calculation (i.e. for relative free energy calculations involving mutation of an atom) and soft-core potentials are employed, similar problems may arise, so it may be useful to remove partial charges on atoms which are being mutated, as well. 

Several groups have developed modified electrostatics scaling methods in an attempt to bypass this problem and allow electrostatics interactions and Lennard-Jones interactions to be turned off in only one set of calculations (for example, Anwar and Heyes{{cite|Anwar2005}}), but since electrostatics transformations are usually so smooth a function of $\lambda$ and need only few $\lambda$ values for good overlap (Shirts et al.{{cite|Shirts2005}} 2005], Mobley et al.{{cite|Mobley2007}}, and others) it is unclear that this results in any significant efficiency gain over performing the transformations separately. 

As noted in Guideline 2, above, electrostatics transformations are typically smooth functions of lambda with good phase-space overlap between even coarsely-spaced lambda values(Shirts et al.{{cite|Shirts2005}} 2005], Mobley et al.{{cite|Mobley2007}}, and others). As a consequence, these are quite efficient compared to Lennard-Jones calculations. As established above, when particles are being inserted or deleted, the electrostatic interactions of these particles should be set to zero before turning off their Lennard-Jones interactions. But what about electrostatic interactions on atoms which are merely being mutated (i.e. a change of partial charge and Lennard-Jones radius), as in relative free energy calculations?
We are not aware of any study which has looked at this in detail, but given the efficiency of free energy calculations modifying electrostatics interactions relative to those significantly modifying Lennard-Jones interactions, we believe it makes sense to perform the two sets of calculations separately. Given that the two transformations have different lambda-dependences, it might actually be less efficient to perform them together than separately. Performing them separately has an additional advantage, as well: Uncertainties in the two components can be assessed separately, and computational effort focused on reducing the largest uncertainty (i.e. by extending some simulations to get additional sampling).
Further testing should be focused in this area, to determine whether alternative scaling approaches which can modify Lennard-Jones and electrostatic interactions simultaneously{{cite|Anwar2005}} are actually more efficient than the approach of separate modification that we propose.

In view of this, our recommendation is that either (a) partial charges on any particles being inserted or deleted be turned off prior to the use of soft core potentials for those particles, or (b) a soft core scheme for electrostatics be implemented to allow simultaneous changes.

## Rules of Thumb for Intermediate States
This section is just the short version of the discussions found elsewhere(mostly [[#Constructing Intermediate States|below]]) on the site. These rules are not the end-all set and you should be familiar with why each one is suggested before just accepting them.

1. **Bonded terms can be modified/turned off linearly.** This includes angle or bond force constants as well as *unconstrained* bond distances.
1. ***Constrained* bonds should not change length.** There are free energy changes that cannot be ignored affiliated with this action.{{cite|Boresch1996}}
1. **Maximize similarity between states by removing/decoupling as few atoms as possible.**
1. ** Do not open and close rings.** This supersedes the previous rule.
1. **Statistical uncertainty between any neighboring states should be equal.** Rather challenging to do, but it has been proven to have the lowest variance path if you can pull it off.{{cite|Shenfeld2009}}
1. **Deleting or adding atoms should always be done with a soft core potential.**
1. ***Changes* in parameters can be done linearly.**
1. **All charge on atoms must be turned off prior to atomic repulsion.** Otherwise you can get an infinite attractive potential and crash your simulation. 
    * Similarly for only changes in terms, it's generally **more efficient to change electrostatic terms separate from Lennard-Jones terms.**
1. **More states is better than fewer.** Variance shrinks rapidly with number of states. You want the difference between intermediaries to be between 2-3 $k_BT$. Obviously you will be limited on CPU power. Fewer states also leads to more samples begin required from each state, so take this into account when deciding number of states as well.  However, for MBAR and TI, it can be shown that spreading samples across multiple states does not significantly affect the uncertainty, since for TI, each state contributes less to the total uncertainty, and in MBAR, data contributes to the statistical precision of states with similar values of lambda.
1. **Shape of the variance does not significantly change with number of atoms, only magnitude.**{{cite|Shirts2003}} More intermediates will still be required for a large number of atoms to reduce statistical noise. 
1. **Charge should be maintained across all $\lambda$.** Simply *having* charged molecules is fine, but the net of the system should remain constant. If you must change the net charge, there are complicated ways to do so.{{cite|Kastenholz2006a}}{{cite|Kastenholz2006b}}
1. **Short prototype simulations are recommended.** Even as short as 100 ps, the prototypes can provide rough magnitude of variance estimates, although will likely under-predict the free energy as many configurations remain unsampled.

## Constructing Intermediate States
This section outlines some of the options one has when constructing their thermodynamic path through the intermediary states. Each method has its own strengths and weakness and should be carefully considered before implementing. Although there is no absolute best way to do this, there are a number of [[#Rules of Thumb for Constructing Intermediate States | best practices]] we strongly consider following, especially for those starting out in free energy calculations. In order to fully understand some of the choices in this section relative to the grand scheme of things, please read the [[Thermodynamic Cycle | thermodynamic cycle]] page.

### Intermolecular Forces
When designing intermediate states, one must decide early on which forces to change, and how to change them. For intermolecular forces, both the electrostatic and Lennard-Jones interactions must be turned off, however, one **should not change all the forces at the same time.** If one were to turn off the repulsive Lennard-Jones components first, unlike charges on atoms would be infinitely attracted to each other and cause the simulation to crash. As such, one should perform one of the following sequences when *decoupling* atoms and/or molecules:

**Decoupling Scheme A**
1. Disable all electrostatics linearly
1. Disable Lennard-Jones terms by soft core

**Decoupling Scheme B**
1. Disable electrostatics AND dispersion (attractive) terms together linearly
1. Disable the repulsive components by soft core

Either method is rather efficient{{cite|Wang2006}}{{cite|Steinbrecher2007}} and can be followed in reverse for appearing molecules. Although all the terms could be turned off by soft core at the same time to avoid negative infinities,{{cite|Wang1994}}{{cite|Blondel2004}} parametrization is a pain and hard to generalize.{{cite|Steinbrecher2011}}

For those just starting, we highly recommend Decoupling Scheme A.

### Topologies
```{figure} ../images/Both_topologies.png
---
width: 300px
name: bothTopologies
---
Single topology (A, upper) and dual topology (B, lower) approaches to constructing an alchemical path between ethane and ethanol. D represents non-interacting dummies, while M represents nonphysical intermediate atoms. In a dual topology approach, no atoms change type, only have their interactions turned off from the rest of the system; however, more atoms need to be altered to go from the initial to the final state. Image Source from a chapter by [http://www.faculty.virginia.edu/shirtsgroup/ Shirts] in Computational Drug Discovery and Design{{cite|Baron2012}}
```

After the intermolecular forces are decided, you will then need to decide the overall method in which you will modify your molecules. There are two main approaches known as the *single topology* approach and the *dual topology* approach; examples of this can be seen in the figure to the side which will be referenced accordingly. Single topology (A, upper) has only one site for any location shared between the end states, and then "dummy" atoms to make up for any unique sites. During the transformation, the dummy atoms are transformed into fully interacting atoms, and the shared site atom is transformed directly to a new atom. Dual topology (B, lower) is different in that shared sites between states do **not** share atoms.  No atom changes its type, just the interactions with the surrounding system.

The dual topology does require more dummies, which means more CPU power and needed intermediates, however, it has a very strong advantage in that the dummies can simultaneously explore more congregational space while decoupled; this perk is amplified when simulations are run with Hamiltonian exchange or expanded ensemble. One may notice that, even though the dummies have nonbonded interactions, they are still "bound" to other atoms which make the end states have slightly nonphysical character. These interactions though can be accounted for by simulating in both vacuum and in molecular surrounds. With fixed bond lengths in the rigid rotor approximation, the effect of these dummies on the free energy is negated.{{cite|Boresch1999}} If the bonds are not constrained, there is a difference, but not enough to be of concern (>0.01 kcal/mol).

In either topology, it may also be necessary to modify the bonded terms (especially for angle and dihedral terms). Because of the time scale of these motions, these terms converge quickly leaving one with small variances but large energy changes. For simple bonded parameters like harmonic spring constants and equilibrium bond lengths, linear changes are perfectly acceptable. *Constrained* bonds are challanging to compute as correction terms are needed from the complete lack of phase space overlap{{cite|Boresch2003}}

Either topological approach will provide the same results and which one is used will often depend on the simulation package/code.

### Alchemically Transforming Rings
Rings in free energy calculations are rather challenging as they prove the exception to the rule of "disappear as few atoms as possible." The act of opening or closing a ring alchemically involves drastic changes to the phase space overlap which should be avoided if at all possible. As such, rings should be disappeared entirely and the new molecule appeared if you find the need to actually break the ring, and in reverse for appearing the ring. This has been shown to be straightforward with the soft core potentials and is still accurate despite the large number of transformations.{{cite|Shirts2005}}{{cite|Shirts2003}}{{cite|Mobley2007}}{{cite|Nicholls2008}}{{cite|Mobley2009}}

### Pulling Methods
A rather different approach to the intermediate states is to set up a system where the ligand is physically pulled away from the protein. The two end states are then where the ligand is in its binding pocket (or nearby at least), and then another where the ligand is far enough away to be considered not interacting with the protein; the free energy in this system can be seen as free energy of binding. Carrying these out can be done with either [[Methods and Simulation Workflow|nonequilibrium methods]],{{cite|Ytreberg2009}}  or by setting up an umbrella simulation with different overlapping harmonic oscillators at increasing distances from the binding pocket, then calculating the PMF.{{cite|Lee2006}}{{cite|Woo2005}}{{cite|Gan2008}}

Although this may seem simple enough, there are a number of problems with pulling methods and so this method is not recommended for those just starting in free energy methods. That said, this method is still valid and has even been suggested to be very efficient for highly charged ligands.{{cite|Woo2005}}

**Challenges for the pulling methods**
* Direct exit path for the ligand from the binding pocket may be difficult to identify. This results in large statistical error.
* Large box sizes are needed for this method to approximate the ligand and protein not interacting.
* If smaller boxes are required (limited computational resources), mean-field or analytical approximations must be made to estimate the "infinite distance away" ligand free energy.

## References
<references>
{{cite|Frenkel2002|Frenkel, D., & Smit, B. (2002). Understanding Molecular Simulation (2nd ed., p. 638). San Diego: Academic Press.}}

{{cite|Beutler1994|Beutler, T., Mark, A., & Schaik, R. van. (1994). Avoiding singularities and numerical instabilities in free energy calculations based on molecular simulations. *Chemical Physics Letters*, 222(June), 529–539.}}

{{cite|Mezei1986|Mezei, M. and Beveridge, D. L. (1986), Free Energy Simulations. Annals of the New York Academy of Sciences, 482: 1–23. doi: 10.1111/j.1749-6632.1986.tb20933.x}}

{{cite|Mruzik1976| Mruzik, M., Abraham, F., Schreiber D., and Pound, G. M., A Monte Carlo study of ion--water clusters, J. Chem. Phys. 64, 481 (1976), DOI:10.1063/1.432264}}

{{cite|Resat1993| Resat, H. and Mezei, M., Studies on free energy calculations. I. Thermodynamic integration using a polynomial path, J. Chem. Phys. 99, 6052 (1993), DOI:10.1063/1.465902}}

{{cite|Zacharias1994|Zacharias, M., Straatsma, T. P., and McCammon, J. A. (1994) Separation-shifted scaling, a new scaling method for Lennard-Jones interactions in thermodynamic integration. *J. Phys. Chem.* 100, 9025–9031.}}

{{cite|Pitera2002|Pitera, J. W., and van Gunsteren, W. F. (2002) A comparison of non-bonded scaling approaches for free energy calculations. *Mol. Simulat.* 28, 45–65.}}

{{cite|Steinbrecher2007|Steinbrecher, T., Mobley, D. L., and Case, D. A. (2007) Nonlinear scaling schemes for Lennard-Jones interactions in free energy calculations. *J. Chem. Phys.* 127, 214108.}}

{{cite|Pearlman1995|Pearlman, D. A., and Connelly, P. R. (1995) Determination of the differential effects of hydrogen bonding and water release on the binding of FK506 to native and TYR82 &rarr; PHE82 FKBP-12 proteins using free energy simulations. *J. Mol. Biol.* 248, 696–717.}}

{{cite|Wang1994|Wang, L., and Hermans, J. (1994) Change of bond length in free-energy simulations: Algorithmic improvements, but when is it necessary? *J. Chem. Phys.* 100, 9129–9139.}}

{{cite|Shirts2005|Shirts, M. R., and Pande, V. S. (2005) Solvation free energies of amino acid side chains for common molecular mechanics water models. *J. Chem. Phys.* 122, 134508.}}

{{cite|Wang2006|Wang, J., Deng, Y., and Roux, B. (2006) Absolute Binding Free Energy Calculations Using Molecular Dynamics Simulations with Restraining Potentials. *Biophys. J.* 91, 2798–2814.}}

{{cite|Pham2001|Pham, T. T., and Shirts, M. R. (2011) Identifying Low Variance Pathways for Free Energy Calculations of Molecular Transformations in Solution Phase. *J. Chem. Phys.* 135, 034114.}}

{{cite|Boresch1996|Boresch, S., and Karplus, M. (1996) The Jacobian factor in free energy simulations. *J. Chem. Phys.* 105, 5145–5154.}}

{{cite|Shenfeld2009|Shenfeld, D. K., Xu, H., Eastwood, M. P., Dror, R. O., and Shaw, D. E. (2009) Minimizing thermodynamic length to select intermediate states for free-energy calculations and replica-exchange simulations. *Phys. Rev. E* 80, 046705.}}

{{cite|Shirts2003|Shirts, M. R., Pitera, J. W., Swope, W. C., and Pande, V. S. (2003) Extremely precise free energy calculations of amino acid side chain analogs: Comparison of common molecular mechanics force fields for proteins. *J. Chem. Phys.* 119, 5740–5761.}}

{{cite|Kastenholz2006a|Kastenholz, M. A., and Hünenberger, P. H. (2006) Computation of methodology-independent ionic solvation free energies from molecular simulations. I. The electrostatic potential in molecular liquids. *J. Chem. Phys.* 124, 124106.}}

{{cite|Kastenholz2006b|Kastenholz, M. A., and Hünenberger, P. H. (2006) Computation of methodology-independent ionic solvation free energies from molecular simulations. II. The hydration free energy of the sodium cation. *J. Chem. Phys.* 124, 224501.}}

{{cite|Blondel2004|Blondel, A. (2004) Ensemble variance in free energy calculations by thermodynamic integration: theory, optimal Alchemical path, and practical solutions. *J. Comput. Chem.* 25, 985–993.}}

{{cite|Steinbrecher2011|Steinbrecher, T., Joung, I., & Case, D. a. (2011). Soft-core potentials in thermodynamic integration: comparing one- and two-step transformations. *J. of Comp. Chem.*, 32(15), 3253–63. doi:10.1002/jcc.21909}}

{{cite|Baron2012|Baron, R. (Ed.). (2012). Computational Drug Discovery and Design (p. 628). New York: Humana Press. doi:10.1007/978-1-61779-465-0}}

{{cite|Boresch1999|Boresch, S., and Karplus, M. (1999) The Role of Bonded Terms in Free Energy Simulations. 2. Calculation of Their Influence on Free Energy differences of Solvation. *J. Phys. Chem.* A 103, 119–136.}}

{{cite|Boresch2003|Boresch, S., Tettinger, F., Leitgeb, M., and Karplus, M. (2003) Absolute binding free energies: A quantitative approach for their calculation. *J. Phys. Chem.* A 107, 9535–9551.}}

{{cite|Mobley2007|Mobley, D. L., Dumont, È., Chodera, J. D., and Dill, K. A. (2007) Comparison of charge models for fixed-charge force fields: Small-molecule hydration free energies in explicit solvent. *J. Phys. Chem.* B 111, 2242–2254.}}

{{cite|Nicholls2008|Nicholls, A., Mobley, D. L., Guthrie, P. J., Chodera, J. D., and Pande, V. S. (2008) Predicting Small-Molecule Solvation Free Energies: An Informal Blind Test for Computational Chemistry. *J. Med. Chem.* 51, 769–779.}}

{{cite|Mobley2009|Mobley, D. L., Bayly, C. I., Cooper, M. D., and Dill, K. A. (2009) Predictions of Hydration Free Energies from All-Atom Molecular Dynamics Simulations?a. *J. Phys. Chem.* B 113, 4533–4537.}}

{{cite|Ytreberg2009|Ytreberg, F. (2009) Absolute FKBP binding affinities obtained via nonequilibrium unbinding simulations. *J. Chem. Phys.* 130, 164906.}}

{{cite|Lee2006|Lee, M. S., and Olson, M. A. (2006) Calculation of Absolute Protein-Ligand Binding Affinity Using Path and Endpoint Approaches. *Biophys. J.* 90, 864–877.}}

{{cite|Woo2005|Woo, H.-J., and Roux, B. (2005) Calculation of absolute protein-ligand binding free energy from computer simulation. *Proc. Natl. Acad. Sci.* 102, 6825–6830.}}

{{cite|Gan2008|Gan, W., and Roux, B. (2008) Binding specificity of SH2 domains: Insight from free energy simulations. *Proteins* 74, 996–1007.}}

{{cite|Anwar2005|Anwar, J. and Heyes, D. M., Robust and accurate method for free-energy calculation of charged molecular systems, *J. Chem. Phys.* 122, 224117 (2005), DOI:10.1063/1.1924449}}
</references>